In [18]:
GYRO = False
SEQUENCE_LENGTH = 4
SEQUENCE_OVERLAP = 3
BATCH_SIZE = 10
EPOCHS = 50
MODEL_NAME = f"physical_activity_(combined_sitting_standing)_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}.keras"

TRAINING_SIZE = 0.8
DEV_SIZE = 0.1
TEST_SIZE = 0.1

# EDIT THE VALUES IN THE CELL ABOVE THEN PRESS RUN ALL

In [19]:
import file_tagger
import sequence_genrator
import tensorflow as tf
import numpy as np
from keras import layers, Sequential
from sklearn.model_selection import train_test_split


In [20]:
DATA_DIRECTORY = "./all_respeck"
PHYSICAL_ACTIVITIES = {
    "sitting&normal_breathing",
    "standing&normal_breathing",
    "lying_down_left&normal_breathing",
    "lying_down_right&normal_breathing",
    "lying_down_back&normal_breathing",
    "lying_down_stomach&normal_breathing",
    "walking&normal_breathing",
    "ascending_stairs&normal_breathing",
    "descending_stairs&normal_breathing",
    "shuffle_walking&normal_breathing",
    "running&normal_breathing",
    "misc_movements&normal_breathing"
}

LABELS = {
    "sitting_or_standing&normal_breathing",
    "lying_down_left&normal_breathing",
    "lying_down_right&normal_breathing",
    "lying_down_back&normal_breathing",
    "lying_down_stomach&normal_breathing",
    "walking&normal_breathing",
    "ascending_stairs&normal_breathing",
    "descending_stairs&normal_breathing",
    "shuffle_walking&normal_breathing",
    "running&normal_breathing",
    "misc_movements&normal_breathing"
}

In [21]:
def generate_training_data(directory, sequence_length, overlap, gyro = GYRO): # if gyro is false, only accelerometer data is used
    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in PHYSICAL_ACTIVITIES:

        # iterates through each csv file for the activity 
        for csv_file in csv_dictionary[key]:
            if gyro:
                sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap)
            else:
                sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap)

            # iterate through each generated sequence
            for sequence in sequences:
                if key == "standing&normal_breathing" or key == "sitting&normal_breathing":
                    key = "sitting_or_standing&normal_breathing"
                tagged_data.append((key, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [22]:
# split data into training, dev, and test sets
def train_dev_test_split(data, labels, dev_size, test_size, random_state=42):
    # Split the data into training and temporary (dev + test) sets
    train_data, temp_data, train_labels, temp_labels = train_test_split(data, labels, test_size=(dev_size + test_size), random_state=random_state)
    
    # Split the temporary data into dev and test sets
    dev_data, test_data, dev_labels, test_labels = train_test_split(temp_data, temp_labels, 
                                                                 test_size=(test_size / (dev_size + test_size)), random_state=random_state)
    
    return train_data, dev_data, test_data, train_labels, dev_labels, test_labels

In [23]:
#CNN MODEL
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        # layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [24]:
tagged_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP)

# Get lists of training data and labels
sequences = [sequence for _, sequence in tagged_sequences]
labels = [label for label, _ in tagged_sequences]


# encode labels to numbers
sequences = np.array(sequences, dtype=np.float32)
label_to_index = {label: idx for idx, label in enumerate(LABELS)}
labels_encoded = [label_to_index[label] for label in labels]
labels_encoded = np.array(labels_encoded)

train_data, dev_data, test_data, train_labels, dev_labels, test_labels = train_dev_test_split(sequences, labels_encoded, dev_size=DEV_SIZE, test_size=TEST_SIZE) #10% dev, 10% test

# train and save model (CHOOSE BETWEEN CNN AND LSTM)
model = train_model_CNN(train_data, train_labels, LABELS, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(dev_data, dev_labels)) #batch_size, epochs


# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the trained model
model.save(f"models/Task1/{MODEL_NAME}_{test_accuracy}")

there are 14878 tagged sequences in the dataset
Epoch 1/50
1191/1191 [==============================] - 8s 6ms/step - loss: 0.8135 - accuracy: 0.7227 - val_loss: 0.4260 - val_accuracy: 0.8609
Epoch 2/50
1191/1191 [==============================] - 6s 5ms/step - loss: 0.3777 - accuracy: 0.8788 - val_loss: 0.2860 - val_accuracy: 0.9099
Epoch 3/50
1191/1191 [==============================] - 5s 4ms/step - loss: 0.2633 - accuracy: 0.9170 - val_loss: 0.1983 - val_accuracy: 0.9368
Epoch 4/50
1191/1191 [==============================] - 6s 5ms/step - loss: 0.2188 - accuracy: 0.9316 - val_loss: 0.1771 - val_accuracy: 0.9489
Epoch 5/50
1191/1191 [==============================] - 5s 4ms/step - loss: 0.1792 - accuracy: 0.9409 - val_loss: 0.1773 - val_accuracy: 0.9409
Epoch 6/50
1191/1191 [==============================] - 5s 4ms/step - loss: 0.1491 - accuracy: 0.9516 - val_loss: 0.1248 - val_accuracy: 0.9651
Epoch 7/50
1191/1191 [==============================] - 5s 4ms/step - loss: 0.1426 - acc

2023-11-06 14:35:47.129808: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,4,128]
	 [[{{node inputs}}]]
2023-11-06 14:35:47.471189: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,4,128]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: models/Task1/physical_activity_(combined_sitting_standing)_epochs:50_batch:10_gyro:False_window:4_overlap:3.keras_0.9778226017951965/assets


INFO:tensorflow:Assets written to: models/Task1/physical_activity_(combined_sitting_standing)_epochs:50_batch:10_gyro:False_window:4_overlap:3.keras_0.9778226017951965/assets
